Analysis

In [648]:
import itertools
import numpy as np
def get_permutations(l,n=3):
    prmts=[]
    for p in itertools. permutations(l, n):
        if sorted(p) not in prmts:
            prmts.append(sorted(p))
    return [ np.asarray(p) for p in prmts if isinstance(p,list) ]
def jsonKeys2int(x):
    if isinstance(x, dict):
            return {int(k):v for k,v in x.items()}
    return x

In [649]:
import pandas as pd

In [915]:
cl=pd.read_json('cl.json').reset_index(drop=True)
cl['sltn']=cl['sltn'].apply(jsonKeys2int)
cl=cl.sort_values('n').reset_index(drop=True)
cl6=pd.read_json('cl6.json').reset_index(drop=True)
cl6['sltn']=cl6['sltn'].apply(jsonKeys2int)
cl6=cl6.sort_values('n').reset_index(drop=True)

DD analysis

In [916]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth',500)

In [1005]:
def extract(l,x):
    for i in range(5):
        try:
            l.remove(x)
        except:
            break
    return l

def remove_list(l,pr):
    for x in pr:
        try:
            l=extract(l,x)
        except:
            pass
    return l

def remove(l,pr):
    if pr:
        pr=np.concatenate(pr).ravel()
    l=remove_list(l,pr)
    return l

def get_massless_fermions(l,ν,m=0,Dim=5):
    if Dim==5:
        d=1
    elif Dim==6:
        d=2
    s=-(ν+m)//d
    l=extract(l,ν)
    if m:
        l=extract(l,m)
    #Dirac masses
    for ss in [s,-s]:
        pr=[ p for  p in get_permuations( l,n=2  ) if np.append( p,ss).sum()==0 ]
        l=remove(l,pr)

    #Majorana masses
    for ss in [s,-s]:
        pr=[ll for ll in l if 2*ll+ss==0]
        l=remove_list(l,pr)
    return list(np.unique(l))

def count_massless_fermions(l,ν,m=0,Dim=5):
    return len( get_massless_fermions(l,ν,m,Dim=Dim) )

In [1006]:
l=[2, -3, -10, 13, 13, -15]
l=[3, -4, -21, 25, 25, -28]
l=[8, 9, 11, -13, -20, -20, 25]
l=[2, 2, -3, 12, -14, -18, 19]
l=[5, 5, 7, -8, -12, -12, 15]
#l=[6, 8, 11, -14, -14, -18, 21]
#l=[3, 3, -8, -15, 20, 20, -23]
#l=[1, -3, 6, -7, -14, 21, 21, -25]
#l=[1, 1, -4, -5, 9, 9, 9, -10, -10]
l=[3, 3, -6, -9, 10, 12, -15, -15, 17]
l=[1, 3, -7, -9, 13, 24, -27, -27, 29]
#XD
l=[2, -5, -5, -5, 7, 8, 8, -10]

In [1013]:
get_massless_fermions([2, -3, -3, -5, 9, -13, 15, 16, -18],ν=-3)==[9]

True

In [1014]:
count_massless_fermions( [2, -5, -5, -5, 7, 8, 8, -10],ν=8,m=0)==2

True

In [1015]:
count_massless_fermions( [2, -5, -5, -5, 7, 8, 8, -10],ν=8,m=-5)==1

True

In [1016]:
count_massless_fermions( [2, -3, -10, 13, 13, -15], ν=13)==0

True

In [1017]:
count_massless_fermions( [5, 13, 13, -16, -18, -23, 26],ν=13)==3

True

In [1018]:
get_massless_fermions( [2, -4, 7, 9, -18, -18, 22],ν=-18 )==[2,7]

True

In [1019]:
# XDirac with two triplets and Dim 6
get_massless_fermions([2, -7, -8, -8, -8, 18, 18, 18, -25],-8,m=18,Dim=6)==[-25]

True

In [1020]:
def massfer(row,label='DarkDirac',nmax=10,Dim=5):        
    # 
    if label=='DarkDirac' or label=='DarkMajor':
        if label=='DarkDirac':
            nuR='DD'
        else:
            nuR='DM'
        if row.get(label)>0:
            cdd=[]
            for ν in row.get(nuR):
                l=row.get('solution').copy()
                cdd.append( count_massless_fermions(l , ν,Dim=Dim)  )
            return min(cdd)
        else:
            return nmax
    if label=='XDirac' or label=='XMajor':
        if label=='XDirac':
            nuR='XD'
        else:
            nuR='XM'
        if row.get(label)>0:
            cddmax=100
            for m in row['sltn'].get(3):
                cdp=[]
                for ν in row.get(nuR):
                    l=row.get('solution').copy()
                    l=[ll for ll in l if ll!=m]
                    cdp.append( count_massless_fermions(l , ν,m,Dim=Dim)  )
                if min(cdp)<cddmax:
                    cdd=cdp
            return min(cdd)
        else:
            return nmax    

In [1021]:
cl['DD_massless']=cl.apply(lambda row: massfer(row,label='DarkDirac'),axis='columns')
cl['DM_massless']=cl.apply(lambda row: massfer(row,label='DarkMajor'),axis='columns')
cl['XD_massless']=cl.apply(lambda row: massfer(row,label='XDirac'),axis='columns')
cl['XM_massless']=cl.apply(lambda row: massfer(row,label='XMajor'),axis='columns')

In [1022]:
cl6['DD_massless']=cl6.apply(lambda row: massfer(row,label='DarkDirac',Dim=6),axis='columns')
cl6['DM_massless']=cl.apply(lambda row: massfer(row,label='DarkMajor',Dim=6),axis='columns')
cl6['DM_massless']=cl6.apply(lambda row: massfer(row,label='DarkMajor',Dim=6),axis='columns')
cl6['XM_massless']=cl6.apply(lambda row: massfer(row,label='XMajor',Dim=6),axis='columns')

In [1023]:
cl['total']=cl['solution'].reset_index(drop=True).apply(lambda l: np.abs( np.array(l) ).sum() )

In [1024]:
cl6['total']=cl6['solution'].reset_index(drop=True).apply(lambda l: np.abs( np.array(l) ).sum() )

In [1025]:
cl[cl.apply(lambda row: massfer(row,label='DarkDirac'),axis='columns')<10][:3]

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,sltn,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total
0,"[-5, 0]","[1, -1]",6,"[2, -3, -10, 13, 13, -15]",2,"[{'D→nu_R+fi+fj': [-15, 2, 13]}, {'D→nu_R+fi+fj': [-10, -3, 13]}]",2,0,0,0,{2: [13]},[13],0,0,0,0,10,10,10,56
1,"[4, 7]","[21, 18]",6,"[3, -4, -21, 25, 25, -28]",7749,"[{'D→nu_R+fi+fj': [-28, 3, 25]}, {'D→nu_R+fi+fj': [-21, -4, 25]}]",2,0,0,0,{2: [25]},[25],0,0,0,0,10,10,10,106
2,"[-4, 0]","[-1, 1]",6,"[3, -5, -12, 17, 17, -20]",1,"[{'D→nu_R+fi+fj': [-20, 3, 17]}, {'D→nu_R+fi+fj': [-12, -5, 17]}]",2,0,0,0,{2: [17]},[17],0,0,0,0,10,10,10,74


In [1026]:
cl[ (cl['n']>0) &  (cl['DarkMajor']>0) ].sort_values(['n','DM_massless','total'])[:3]

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,sltn,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total
81,"[1, 0, -1]","[-4, -6]",7,"[1, 1, -3, -4, 6, 6, -7]",3,"[{'D→nu_R+fi+fj': [-7, 1, 6]}, {'D→nu_R+2fi': [6, -3]}]",1,1,0,0,"{2: [1, 6]}","[1, 6]",[6],0,0,1,1,10,10,28
57,"[-1, -4, -3]","[2, 0]",7,"[1, 3, -4, 5, -6, -6, 7]",3,"[{'D→nu_R+2fi': [-6, 3]}, {'D→nu_R+fi+fj': [-6, 1, 5]}]",1,1,0,0,{2: [-6]},[-6],[-6],0,0,2,2,10,10,32
10,"[-5, -9, 7]","[5, 15]",7,"[2, -4, 7, 9, -18, -18, 22]",625,"[{'D→nu_R+2fi': [-18, 9]}, {'D→nu_R+fi+fj': [-18, -4, 22]}]",1,1,0,0,{2: [-18]},[-18],[-18],0,0,2,2,10,10,80


In [1027]:
cl[ (cl['n']==9) &  (cl['DarkDirac']>0) ].sort_values(['DD_massless','total'])[:3]

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,sltn,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total
3735,"[4, 8, 6, 3]","[-2, 3, 0]",9,"[1, 1, 2, 2, 3, -5, -6, -6, 8]",4,"[{'D→nu_R+fi+fj': [-5, 2, 3]}, {'D→nu_R+2fi': [-6, 3]}]",1,1,0,0,"{2: [1, 2, -6]}",[2],[-6],0,0,0,4,10,10,34
3388,"[1, 0, -5, 9]","[-3, -1, -2]",9,"[1, -2, 3, 4, 6, -7, -7, -7, 9]",2,"[{'D→nu_R+fi+fj': [-7, -2, 9]}, {'D→nu_R+fi+fj': [-7, 1, 6]}, {'D→nu_R+fi+fj': [-7, 3, 4]}]",3,0,0,0,{3: [-7]},[-7],0,0,0,0,10,10,10,46
3334,"[4, 10, 14, 3]","[16, 2, 10]",9,"[1, -3, -3, -4, -5, 8, 9, 9, -12]",1272,"[{'D→nu_R+fi+fj': [-5, -3, 8]}, {'D→nu_R+fi+fj': [-5, -4, 9]}]",2,0,0,0,"{2: [-3, 9]}","[-3, 9]",0,0,0,0,10,10,10,54


def repeated(f):
    r={}
    r[1]=f
    for ri in range(2,f.size+2):
        r[ri]=r[ri-1][pd.Series(r[ri-1]).duplicated()]
        if ri>2:
            for a in np.unique(r[ri]):
                r[ri-1]=r[ri-1][ r[ri-1]!=a ]
                if len(r[ri-1])>0:
                    r[ri-1][ r[ri-1]!=a ]
        if len(r[ri])<=1:
            r[ri]=r[ri]
            r.pop(1)
            break
    return dict( (k,list(v)) for k,v in r.items() if len(v)!=0 )

In [678]:
cl[2799:2800]['solution'].apply(lambda s: repeated(np.asarray(s)))

2799    {2: [-23], 3: [2]}
Name: solution, dtype: object

In [679]:
r={2: [-23], 3: [2]} 

In [683]:
dict( ((k,v) for k,v in r.items() if k!=3) )

{2: [-23]}

In [645]:
def get_nuR_i(row,i,model,label):
    print( row[model]>0,'**',  row['sltn'].get(i))
    if row[model]>0 and row['sltn'].get(i) and row['nu_R'] and len( [ d.get(label) for d in row['nu_R'] if d.get(label)] )>0:
        M=[m  for m in  row['nu_R'] if m.get(label)]
        nuR=[n for n in row['sltn'][i] for m in M  if n in m.get(label)  ]
        #print(nuR)
        if nuR:
            return list(np.unique(nuR))
    else:
        return []

In [647]:
kk.apply(lambda row: get_nuR_i(row,3,'DarkDirac','D→nu_R+fi+fj'),axis='columns')

True ** None


1515    []
dtype: object

In [601]:
kk.sltn.loc[1542]['4']

[2]

In [640]:
cl[1537:1538]

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,sltn,DD,DM,XD,XM
1537,"[6, -1, 7, 1]","[12, 0, 9]",9,"[2, -3, -3, -5, 9, -13, 15, 16, -18]",432,"[{'D→nu_R+fi+fj': [-13, -3, 16]}]",1,0,0,0,{2: [-3]},[-3],0,0,0


In [471]:
cl[:3]

,DarkDirac,DarkMajor,XDirac,XMajor,gcd,k,l,n,nu_R,sltn,solution,DD,DM,XD,XM
0,2,0,0,0,2,"[-5, 0]","[1, -1]",6,"[{'D→nu_R+fi+fj': [-15, 2, 13]}, {'D→nu_R+fi+fj': [-10, -3, 13]}]",{'2': [13]},"[2, -3, -10, 13, 13, -15]",[13],0,0,0
1,2,0,0,0,7749,"[4, 7]","[21, 18]",6,"[{'D→nu_R+fi+fj': [-28, 3, 25]}, {'D→nu_R+fi+fj': [-21, -4, 25]}]",{'2': [25]},"[3, -4, -21, 25, 25, -28]",[25],0,0,0
2,2,0,0,0,1,"[-4, 0]","[-1, 1]",6,"[{'D→nu_R+fi+fj': [-20, 3, 17]}, {'D→nu_R+fi+fj': [-12, -5, 17]}]",{'2': [17]},"[3, -5, -12, 17, 17, -20]",[17],0,0,0


### Tree level neutrino Dirac masses

In [416]:
l=get_massless_fermions( [2, -3, -10, 13, 13, -15], ν=13,m=0)
[ ll for ll in l if ll+ν-s==0]

[]

In [417]:
[ ll for ll in l if ll-ν+s==0]

[]